# Film Analysis Agent with Strands Agents SDK

## Overview
In this notebook, we'll rebuild the Film Agent using the [Strands Agents SDK](https://github.com/strands-agents/sdk-python?tab=readme-ov-file), an open source framework that takes a model-driven approach to building and running AI agents in just a few lines of code. Strands simplifies agent development by embracing the capabilities of state-of-the-art models to plan, chain thoughts, call tools, and reflect.

We'll implement the same functionality as in the original notebook but using the Strands Agents SDK, which is already used in production by several AWS teams including Amazon Q Developer, AWS Glue, and VPC Reachability Analyzer.

We'll explore:
1. Creating an agent with knowledge base integration
2. Implementing direct function integration for celebrity detection
3. Setting up MCP-based tools for better separation of concerns

<div class="alert-warning">
    <b>Warning:</b> you need to complete module 2 first because some resources like knowledge base are created in that module.
</div>

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import os

# Import strands-agents components
from strands import Agent, tool
from strands.tools.mcp import MCPClient
from strands_tools import use_llm, memory
from mcp import stdio_client, StdioServerParameters
import json_repair

# Initialize AWS clients
boto3_session = boto3.Session()
dynamodb_resource = boto3_session.resource('dynamodb')
rek_client = boto3_session.client('rekognition')
bedrock_agent_runtime = boto3_session.client('bedrock-agent-runtime')

## Loading Configuration

First, let's load the configuration from the previous modules, including knowledge base name, agent instructions, and other parameters.

In [ ]:
%store -r knowledge_base_name
%store -r kb_config
%store -r agent_instruction
%store -r agent_name
%store -r video_analysis
%store -r cast_table
%store -r cast_pk
%store -r film_video_s3_path

## Multiple Model Providers
Strands SDK takes a model-driven approach and offers flexible model support. It can run anywhere and supports any model with reasoning and tool use capabilities, including models from Amazon Bedrock, Anthropic, Meta, Ollama, and other providers through LiteLLM.

Like the two strands of DNA, Strands connects two core pieces of the agent together: the model and the tools. Strands plans the agent's next steps and executes tools using the advanced reasoning capabilities of models.

<div class="alert-info">
    <b>INFO:</b> Uncomment the code below to try other model providers. Please remember to set your API key in the environment variable
</div>


In [ ]:
from strands.models import BedrockModel
# from strands.models.ollama import OllamaModel
# from strands.models.llamaapi import LlamaAPIModel

# Bedrock
bedrock_model = BedrockModel(
  model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
  temperature=0.1,
)
agent = Agent(model=bedrock_model)
agent("Tell me about Agentic AI")

# # Ollama
# ollama_modal = OllamaModel(
#   host="http://localhost:11434",
#   model_id="llama3"
# )
# agent = Agent(model=ollama_modal)
# agent("Tell me about Agentic AI")

# # Llama API
# llama_model = LlamaAPIModel(
#     model_id="Llama-4-Maverick-17B-128E-Instruct-FP8",
# )
# agent = Agent(model=llama_model)
# response = agent("Tell me about Agentic AI")

## Python-Based Tools

In Strands' model-driven approach, tools are key to how you customize the behavior of your agents. Tools can retrieve relevant documents from a knowledge base, call APIs, run Python logic, or return static strings with additional model instructions.

You can easily define tools using Python decorators or just leveraging existing tools available, which is one of the core concepts of Strands Agents. Compared with frameworks that require complex workflows, Strands simplifies agent development by letting the model drive tool selection and execution.

### Creating a Basic Agent with Knowledge Base

Now, let's create a basic agent that can answer questions using the knowledge base.

In [ ]:
# Set an environment variable for kb
os.environ["STRANDS_KNOWLEDGE_BASE_ID"] = kb_config['kb_id']

In [ ]:
query = "which film is directed by Curtis Clark"

@tool
def retrieve_kb(query:str)->str:
    """Process a user query with the knowledge base agent."""
    agent = Agent(tools=[memory, use_llm])

    # Optimized retrieval parameters
    results = agent.tool.memory(
        action="retrieve", 
        query=query,
        min_score=0.4,  # Set minimum relevance threshold
        max_results=1   # Limit number of results
    )

    print(f"Retrieved results: {str(results)}")

    # System prompt for generating answers from retrieved information
    ANSWER_SYSTEM_PROMPT = """
    You are a helpful knowledge assistant that provides clear, concise answers 
    based on information retrieved from a knowledge base.
    """

    # Convert the result to a string to extract just the content text
    result_str = str(result)
    # Generate a clear, conversational answer using the retrieved information
    return agent.tool.use_llm(
        prompt=f"User question: \"{query}\"\n\nInformation from knowledge base:\n{result_str}\n\nStart your answer with newline character and just answer the question directly:",
        system_prompt=ANSWER_SYSTEM_PROMPT
    )

retrieve_kb(query)

### Create another tool to look up cast and role

Let's implement another tool that can look up celebrites's role in the movie from dynamoDB.

In [ ]:
@tool
def get_cast_member(cast_id):
    """Query DynamoDB to get cast member information."""
    try:
        table = dynamodb_resource.Table(cast_table)
        key_expression = Key(cast_pk).eq(cast_id)
        query_data = table.query(
                KeyConditionExpression=key_expression
            )
        return query_data['Items']
    except Exception as e:
        print(f'Error querying table: {cast_table}.')
        print(str(e))

In [ ]:
agent = Agent(model=bedrock_model, tools=[retrieve_kb, get_cast_member])
response = agent("Can you tell me Kevin Kilner's role (id: 4kn3Xu8r) in th film `Meridian`?")

## Built-in MCP Support
Strands SDK provides native support for Model Context Protocol (MCP) servers, enabling access to thousands of published MCP servers that can be used as tools for your agent. This is particularly useful for complex agent use cases where you need specialized tools.

Strands has published its own MCP server that you can use with any MCP-enabled development tool, such as the Q Developer CLI or Cline. This makes it easy to start building new agents today with your favorite AI-assisted development tool.

You can also easily create your own MCP server by preparing a MCP server Python script like below.

In [ ]:
%pycat key_figures_mcp_server.py

### Setting Up MCP Client

Now, let's set up the MCP client to connect to our server.

<div class="alert-warning">
    <b>Warning:</b> This MCP server may take up to 5 minutes to run. because of rekignition video API for celebrity detection.
</div>

In [ ]:
# Get AWS credentials for the MCP server
def get_credentials():
    credentials = boto3_session.get_credentials()
    frozen_creds = credentials.get_frozen_credentials()
    region = boto3_session.region_name
    
    return {
        "AWS_ACCESS_KEY_ID": frozen_creds.access_key,
        "AWS_SECRET_ACCESS_KEY": frozen_creds.secret_key,
        "AWS_DEFAULT_REGION": region
    }

For this query, the agent will first look up the film, and then use the informaiton to detect the celebrity and lookup their role in the film.

In [ ]:
# Step 1: Define MCP stdio parameters
server_params = StdioServerParameters(
    command="python",
    args=[
        "key_figures_mcp_server.py",
    ],
    env={"CAST_TABLE": cast_table, "CAST_PK":cast_pk, **get_credentials()},
)
# Create MCP client
mcp_client = MCPClient(
    lambda: stdio_client(server_params)
)

with mcp_client:
   agent = Agent(tools=[retrieve_kb] + mcp_client.list_tools_sync())
   response = agent(f"""
    Here is a clip extraction:
    {video_analysis}

    if needed, here is the s3 location of the video:
    {film_video_s3_path}
    can you tell me which film is this clip from?
    """)
   additional_info = response.message['content'][0]['text'].strip()
   print(additional_info)


In [ ]:
%store additional_info

## Conclusion

In this notebook, we've demonstrated how to use the Strands Agents SDK to build a film analysis agent with the same functionality as the original implementation. We've shown:

1. **Basic Agent with Knowledge Base**: Using Strands to create an agent that can answer questions about films using a knowledge base.

2. **Direct Function Integration**: Implementing celebrity detection as a direct tool function that the agent can use.

3. **MCP Integration**: Setting up an MCP server and client to provide the same functionality with better separation of concerns.

The Strands Agents SDK provides a clean, intuitive interface for building AI agents that scales from simple to complex use cases, and from local development to deployment in production. Running agents in production is a key tenet for the design of Strands, which includes a deployment toolkit with reference implementations to help you take your agents to production.

Strands is flexible enough to support a variety of architectures in production, whether you want to run your agent locally, behind an API, or in a distributed environment with tools running in separate backends.